In [1]:
import json, re
from collections import defaultdict, Counter

In [2]:
# 1000 normostran
HAHAKOEFICIENT = 1800 * 1000

In [3]:
def normalize_name(name):
    prefixlist = ["Poslanec", "Poslankyně", "Předseda PSP", "Předsedkyně PSP", "Místopředseda PSP", "Místopředsedkyně PSP",
                  "Předseda vlády ČR", "Prezident České republiky", "Evropská komisařka pro spravedlnost, spotřebitele a rovnost žen a mužů",
                  "Guvernér ČNB", "Veřejná ochránkyně práv", "Senátorka", "Senátor", "Předsedající",
                  "Místopředseda vlády, ministr vnitra a ministr zahraničních věcí ČR",
                  "Místopředseda vlády ČR, ministr vnitra a ministr zahraničních věcí"]
    for el in prefixlist:
        if name.startswith(el): return name.replace(el + " ", "")
    if name == " Radek Vondráček": return "Radek Vondráček"
    name = re.sub("Ministr(yně)? .* ČR ", "", name)
    name = re.sub("Místopředsed(?:a|kyně) vlády(?: ČR)?(?: a ministr(?:yně)? .*?)?(?: ČR)? ([A-Z])", "\\1", name)
    return name

In [4]:
with open("wc.json", encoding="utf8") as file:
    data = json.load(file)

In [5]:
with open("data.json", encoding="utf8") as file:
    smich_data = json.load(file)

In [6]:
smich_counter = defaultdict(int)
for smich in smich_data:
    smich_count = len(re.findall("\(.*?[sS]mích.*?\)", smich["speech"]))
    smich_counter[normalize_name(smich["speaker"])] += smich_count

In [9]:
sum(smich_counter.values())

1903

In [40]:
#join
for name in smich_counter:
    for idx, el in enumerate(data):
        if el["name"] == name:
            data[idx]["haha"] = smich_counter[name]

In [41]:
#filter
data = [el for el in data if "haha" in el]

In [42]:
#normalize
for idx, el in enumerate(data):
    data[idx]["hahaIndex"] = round(el["haha"] / el["wc"] * HAHAKOEFICIENT)

In [43]:
data.sort(key=lambda x: x["hahaIndex"], reverse=True)

In [44]:
with open("results_hahaindex.json","w",encoding="utf8") as file:
    json.dump([el for el in data if el["haha"] > 2], file, ensure_ascii=False)